In [5]:
import platform
from dymola.dymola_interface import DymolaInterface
from dymola.dymola_exception import DymolaException
from modelicares import SimRes
import pandas as pd
import numpy as np
import os
import shutil

In [198]:
#This is intended to be used in the manuelnvro Dell using Dymola 2020

In [6]:
#Setting Dymola Interface
dymola = DymolaInterface("/opt/dymola-2020-x86_64/bin64/dymola.sh")
#Setting OpenIPSL library
dymola.openModel("/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/OpenIPSL-master/OpenIPSL/package.mo") 
print("Dymola Machines Simulation Start...\n")

Dymola Machines Simulation Start...



In [7]:
#Creation of matrix with names, paths and variables
machines = { 'names' : ["GENROU","GENSAL", "GENCLS", "GENROE", "GENSAE", "CSVGN1"],
            'path' : ["OpenIPSL.Examples.Machines.PSSE.GENROU", "OpenIPSL.Examples.Machines.PSSE.GENSAL", "OpenIPSL.Examples.Machines.PSSE.GENCLS", "OpenIPSL.Examples.Machines.PSSE.GENROE", "OpenIPSL.Examples.Machines.PSSE.GENSAE", "OpenIPSL.Examples.Machines.PSSE.CSVGN1"],
            'delta' : ['gENROU.delta', 'gENSAL.delta', 'gENCLS.delta', 'gENROE.delta', 'gENSAE.delta', 'cSVGN1.delta'],
           'pelec' : ['gENROU.PELEC', 'gENSAL.PELEC', 'gENCLS.PELEC', 'gENROE.PELEC', 'gENSAE.PELEC', 'cSVGN1.PELEC'],
           'speed' : ['gENROU.SPEED', 'gENSAL.SPEED', 'gENCLS.SPEED', 'gENROE.SPEED', 'gENSAE.SPEED', 'cSVGN1.SPEED']}


In [8]:
#Delete old results
shutil.rmtree('/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/')
#Create Exciters folder
os.makedirs('/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/')
os.chdir(f"/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/")
for machineNumber, machineName in enumerate(machines['names']):
    os.makedirs(f'{machineName}')

In [9]:
#For loop that will iterate between machines, simulate, and create the .csv file
for machineNumber, machineName in enumerate(machines['names']):
    try:
        print(f"{machineName} Simulation Start...")
        dymola.cd("/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/" + machineName)
        resultPath = f"/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/{machineName}/" + machineName
        result = dymola.simulateModel(machines['path'][machineNumber], 
                                stopTime=10.0,
                                numberOfIntervals = 5000,
                                resultFile = resultPath)
        if not result:
            print("Simulation failed or model was not found. Below is the translation log:\n")
            log = dymola.getLastErrorLog()
            print(log)
        else:
            print(f"{machineName} Simulation OK...")
            print(".csv Writing Start...")
            #Selecting Result File
            sim = SimRes(f"/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/{machineName}/{machineName}.mat")
            #Selecting Variables
            variables = ['Time', machines['delta'][machineNumber], machines['pelec'][machineNumber], machines['speed'][machineNumber], 'GEN1.V', 'LOAD.V', 'GEN2.V', 'FAULT.V' ]
            df_variables = pd.DataFrame([], columns = variables)
            for var in variables:
                df_variables.drop(var, axis = 1, inplace = True)
                #Change from Radians to Degrees
                if var == machines['delta'][machineNumber]:
                    df_variables[var] = np.array(sim[var].values()*(180/np.pi))    
                else:
                    df_variables[var] = np.array(sim[var].values())
            print(f"{machineName} Variables OK...")
            #Changing current directory
            os.chdir(f"/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/")
            df_variables.to_csv(f'{machineName}.csv', index = False)          
            print(f"{machineName} Write OK...")       
        try:
            shutil.rmtree(f"/home/manuelnvro/dev/Gitted/NYPAModelTransformation/OpenIPSLVerification/VerficationRoutines/Dymola/WorkingDir/Dymola/Machines/{machineName}/")
            print("Delete OK...\n")
        except:
            pass          
    except DymolaException as ex:
        print("Error: " + str(ex))
print('Machine Examples Simulation OK...')

GENROU Simulation Start...
GENROU Simulation OK...
.csv Writing Start...
GENROU Variables OK...
GENROU Write OK...
Delete OK...

GENSAL Simulation Start...
GENSAL Simulation OK...
.csv Writing Start...
GENSAL Variables OK...
GENSAL Write OK...
Delete OK...

GENCLS Simulation Start...
Simulation failed or model was not found. Below is the translation log:

Translation of <a href="Modelica://OpenIPSL.Examples.Machines.PSSE.GENCLS">OpenIPSL.Examples.Machines.PSSE.GENCLS</a>:
Did not find model OpenIPSL.Examples.Machines.PSSE.GENCLS
Error: ERRORS have been issued.

Delete OK...

GENROE Simulation Start...
GENROE Simulation OK...
.csv Writing Start...
GENROE Variables OK...
GENROE Write OK...
Delete OK...

GENSAE Simulation Start...
GENSAE Simulation OK...
.csv Writing Start...
GENSAE Variables OK...
GENSAE Write OK...
Delete OK...

CSVGN1 Simulation Start...
Simulation failed or model was not found. Below is the translation log:

Translation of <a href="Modelica://OpenIPSL.Examples.Machine